<a href="https://colab.research.google.com/github/fritzmartin003/RAG-System-Projekt/blob/main/RAG_System_Projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymupdf openai numpy scipy


In [ ]:
#mit Secret Manager in Google Colab
from google.colab import userdata
userdata.get('rag-key')


In [ ]:
pdf_path = "SakowskiBuch.pdf"

In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text("text") + "\n"
    return text

# Text aus PDF extrahieren
pdf_text = extract_text_from_pdf(pdf_path)
print(" Text aus PDF extrahiert!")


 Text aus PDF extrahiert!


In [ ]:
def split_text(text, chunk_size=500, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap  # Überschneidung
    return chunks

chunks = split_text(pdf_text)
print(f" PDF in {len(chunks)} Chunks unterteilt!")


 PDF in 1559 Chunks unterteilt!


In [ ]:
import numpy as np
import openai
import os

In [ ]:
# API-Schlüssel aus der Umgebungsvariable lesen
openai.api_key = os.getenv('rag')

# Funktion, um Embeddings für einen Text zu holen
def get_embedding(text):
    response = openai.Embedding.create(
        model="gpt-4o-mini",  # OpenAI Embedding Modell
        input=text
    )
    return np.array(response["data"][0]["embedding"])

# Alle Chunks in Embeddings umwandeln
chunk_embeddings = np.array([get_embedding(chunk) for chunk in chunks])
print(" Chunks als Vektoren gespeichert!")


AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.

In [ ]:
from scipy.spatial.distance import cosine

def get_relevant_chunks(query, top_k=3):
    query_embedding = get_embedding(query)

    # Cosine Similarity berechnen
    similarities = [1 - cosine(query_embedding, chunk_emb) for chunk_emb in chunk_embeddings]

    # Top-k Chunks auswählen (die mit der höchsten Ähnlichkeit)
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    return [chunks[i] for i in top_indices]

print(" Ähnlichkeitssuche bereit!")


 Ähnlichkeitssuche bereit!


In [ ]:
def ask_pdf(query):
    relevant_chunks = get_relevant_chunks(query)  # Die relevanten Chunks finden
    context = "\n".join(relevante_chunks)  # Den Kontext vorbereiten

    prompt = f"""
    Beantworte die folgende Frage basierend auf diesem Textausschnitt:
    {context}

    Frage: {query}
    """

    # Anfrage an GPT-4 senden
    response = openai.ChatCompletion.create(
        model="gpt-4",  # OpenAI GPT-4 Modell
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message['content']  # Die Antwort extrahieren

# Beispiel-Frage
frage = "Was ist die Hauptaussage des Buches?"
antwort = ask_pdf(frage)
print("Antwort:", antwort)


AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.